In [1]:
import re
import csv
import time
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [30]:
url ='https://toto.rakuten.co.jp/toto/result/1249/'

In [31]:
r = requests.get(url)
r

<Response [200]>

In [32]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ja" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="第1249回のtotoくじ結果、当せん結果をチェック！楽天totoならBIG、totoをインターネットでいつでも購入。キャリーオーバーで最高6億円のくじ！" name="description"/>
<meta content="第1249回,toto,当せん,結果,BIG,トト,ビッグ,楽天,rakuten,購入" name="keywords"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="第1249回 totoくじ結果：楽天toto" id="meta_title" property="og:title">
<meta content="第1249回 totoくじ結果、当せん結果をチェック！楽天totoならBIG、totoをインターネットでいつでも購入。キャリーオーバーで最高6億円のくじ！" id="meta_description" property="og:description">
<meta content="楽天toto" property="og:site_name">
<meta content="https://toto.rakuten.co.jp/img/other/ogt_toto.gif" property="og:image"/>
<meta content="sports_league" property="og:type"/>
<title>楽天toto: 第1249回 totoくじ結果</title>
<link href="/favicon.ico" rel="shortcut icon"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon-precomposed"/>
<link hr

In [33]:
wining = soup.select("#minitotoBlockA .tbl-basic-money tbody td")[0].string
wining

'5,800円'

In [6]:
date = soup.select("#minitotoBlockA .tbl-basic-day td")[0].string
date

'2021年7月20日(火)'

In [7]:
match = soup.select("#minitotoBlockA .tbl-result tbody tr")
match

[<tr class="vary">
 <td class="number">1</td>
 <td>清水</td>
 <td>0－2</td>
 <td>川崎</td>
 <td class="empty"><img alt="" height="1" src="/toto/result/images/t.gif" width="1"/></td>
 <td class="result">2</td>
 </tr>,
 <tr>
 <td class="number">2</td>
 <td>福岡</td>
 <td>0－1</td>
 <td>Ｇ大阪</td>
 <td class="empty"><img alt="" height="1" src="/toto/result/images/t.gif" width="1"/></td>
 <td class="result">2</td>
 </tr>,
 <tr class="vary">
 <td class="number">3</td>
 <td>Ｃ大阪</td>
 <td>1－1</td>
 <td>神戸</td>
 <td class="empty"><img alt="" height="1" src="/toto/result/images/t.gif" width="1"/></td>
 <td class="result">0</td>
 </tr>,
 <tr class="middleGroup even">
 <td class="number">4</td>
 <td>鳥栖</td>
 <td>3－1</td>
 <td>名古屋</td>
 <td class="empty"><img alt="" height="1" src="/toto/result/images/t.gif" width="1"/></td>
 <td class="result">1</td>
 </tr>,
 <tr class="middleGroup">
 <td class="number">5</td>
 <td>千葉</td>
 <td>2－1</td>
 <td>金沢</td>
 <td class="empty"><img alt="" height="1" src="/toto/resu

In [8]:
match[1].select("td")

[<td class="number">2</td>,
 <td>福岡</td>,
 <td>0－1</td>,
 <td>Ｇ大阪</td>,
 <td class="empty"><img alt="" height="1" src="/toto/result/images/t.gif" width="1"/></td>,
 <td class="result">2</td>]

In [9]:
[match[1].select("td")[1].string, match[1].select("td")[3].string, match[1].select("td")[5].string]

['福岡', 'Ｇ大阪', '2']

In [10]:
date = soup.select("#minitotoBlockA .tbl-basic-day td")[0].string
wining = soup.select("#minitotoBlockA .tbl-basic-money tbody td")[0].string
toto_data = [435,"A",date,wining]
for i in range(5):
    match_data =[match[i].select("td")[1].string, match[i].select("td")[3].string, match[i].select("td")[5].string]
    toto_data.extend(match_data)



In [11]:
toto_data #1行分のデータ

[435,
 'A',
 '2021年7月20日(火)',
 '5,800円',
 '清水',
 '川崎',
 '2',
 '福岡',
 'Ｇ大阪',
 '2',
 'Ｃ大阪',
 '神戸',
 '0',
 '鳥栖',
 '名古屋',
 '1',
 '千葉',
 '金沢',
 '1']

In [12]:
column_array=[["No","Group","Date","Winning","Match1","Match1","Match1","Match2","Match2","Match2","Match3","Match3","Match3","Match4","Match4","Match4","Match5","Match5","Match5"],["","","","","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L"]]

In [13]:
column_array

[['No',
  'Group',
  'Date',
  'Winning',
  'Match1',
  'Match1',
  'Match1',
  'Match2',
  'Match2',
  'Match2',
  'Match3',
  'Match3',
  'Match3',
  'Match4',
  'Match4',
  'Match4',
  'Match5',
  'Match5',
  'Match5'],
 ['',
  '',
  '',
  '',
  'Home',
  'Away',
  'W/L',
  'Home',
  'Away',
  'W/L',
  'Home',
  'Away',
  'W/L',
  'Home',
  'Away',
  'W/L',
  'Home',
  'Away',
  'W/L']]

In [14]:
tuples = list(zip(*column_array))
tuples

[('No', ''),
 ('Group', ''),
 ('Date', ''),
 ('Winning', ''),
 ('Match1', 'Home'),
 ('Match1', 'Away'),
 ('Match1', 'W/L'),
 ('Match2', 'Home'),
 ('Match2', 'Away'),
 ('Match2', 'W/L'),
 ('Match3', 'Home'),
 ('Match3', 'Away'),
 ('Match3', 'W/L'),
 ('Match4', 'Home'),
 ('Match4', 'Away'),
 ('Match4', 'W/L'),
 ('Match5', 'Home'),
 ('Match5', 'Away'),
 ('Match5', 'W/L')]

In [15]:
column = pd.MultiIndex.from_tuples(tuples)
column

MultiIndex([(     'No',     ''),
            (  'Group',     ''),
            (   'Date',     ''),
            ('Winning',     ''),
            ( 'Match1', 'Home'),
            ( 'Match1', 'Away'),
            ( 'Match1',  'W/L'),
            ( 'Match2', 'Home'),
            ( 'Match2', 'Away'),
            ( 'Match2',  'W/L'),
            ( 'Match3', 'Home'),
            ( 'Match3', 'Away'),
            ( 'Match3',  'W/L'),
            ( 'Match4', 'Home'),
            ( 'Match4', 'Away'),
            ( 'Match4',  'W/L'),
            ( 'Match5', 'Home'),
            ( 'Match5', 'Away'),
            ( 'Match5',  'W/L')],
           )

In [16]:
df=pd.DataFrame(columns=column)
df

Empty DataFrame
Columns: [(No, ), (Group, ), (Date, ), (Winning, ), (Match1, Home), (Match1, Away), (Match1, W/L), (Match2, Home), (Match2, Away), (Match2, W/L), (Match3, Home), (Match3, Away), (Match3, W/L), (Match4, Home), (Match4, Away), (Match4, W/L), (Match5, Home), (Match5, Away), (Match5, W/L)]
Index: []

In [17]:
df.loc["435_A"]= toto_data


In [18]:
df

No Group           Date Winning Match1          Match2           \
                                          Home Away W/L   Home Away W/L   
435_A  435     A  2021年7月20日(火)  5,800円     清水   川崎   2     福岡  Ｇ大阪   2   

      Match3          Match4          Match5           
        Home Away W/L   Home Away W/L   Home Away W/L  
435_A    Ｃ大阪   神戸   0     鳥栖  名古屋   1     千葉   金沢   1

In [139]:
column_array=[["No","Group","Date","Winning","Match1","Match1","Match1","Match2","Match2","Match2","Match3","Match3","Match3","Match4","Match4","Match4","Match5","Match5","Match5"],["","","","","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L"]]
column_tuples = list(zip(*column_array))
column = pd.MultiIndex.from_tuples(column_tuples)
df=pd.DataFrame(columns=column)
club_id = pd.read_csv("./club_and_id.csv")

for n in range(1240,1250):
    url =f'https://toto.rakuten.co.jp/toto/result/{n}/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        for group in ["A","B"]:
            date = soup.select(f"#minitotoBlock{group} .tbl-basic-day td")[0].string
            wining = soup.select(f"#minitotoBlock{group} .tbl-basic-money tbody td")[0].string
            toto_data = [n, group, date,wining]

            match = soup.select(f"#minitotoBlock{group} .tbl-result tbody tr")
            for i in range(5):
                home_ja = match[i].select("td")[1].string
                away_ja = match[i].select("td")[3].string

                home = ""
                away = ""

                for index,row in club_id.iterrows():
                    if home_ja == row["club_ja"]:
                        home = row["club"]
                    elif away_ja == row["club_ja"]:
                        away = row["club"]

                match_data =[home, away, match[i].select("td")[5].string]
                toto_data.extend(match_data)

            if not "" in toto_data:
                df.loc[f"{n}_{group}"]= toto_data
    
    except IndexError:
            pass
    
    time.sleep(1)

In [140]:
df

No Group           Date  Winning               Match1  \
                                                           Home   
1240_A  1240     A  2021年5月18日(火)  35,220円       vegalta-sendai   
1240_B  1240     B  2021年5月18日(火)   6,930円      shimizu-s-pulse   
1241_A  1241     A  2021年5月25日(火)  34,450円           sagan-tosu   
1242_A  1242     A   2021年6月1日(火)  11,700円  yokohama-fa-marinos   
1245_A  1245     A  2021年6月22日(火)  21,340円       avispa-fukuoka   
1246_A  1246     A  2021年6月29日(火)   1,970円     v-varen-nagasaki   
1247_A  1247     A   2021年7月6日(火)  13,800円     matsumoto-yamaga   
1248_A  1248     A  2021年7月13日(火)  34,790円  yokohama-fa-marinos   

                                           Match2                           \
                        Away W/L             Home                 Away W/L   
1240_A        avispa-fukuoka   2  kashima-antlers  yokohama-fa-marinos   1   
1240_B  nagoya-grampus-eight   2      yokohama-fc      shonan-bellmare   1   
1241_A       kashima-antlers   1   vegalta-sendai         oita-trinita   1   
1242_A       shimizu-s-pulse   1         fc-tokyo  sanfrecce-hiroshima   0   
1245_A           vissel-kobe   2      yokohama-fc             fc-tokyo   2   
1246_A          jubilo-iwata   2      yokohama-fc      shimizu-s-pulse   0   
1247_A           tokyo-verdy   1      vissel-kobe      shonan-bellmare   1   
1248_A        avispa-fukuoka   1     oita-trinita   urawa-red-diamonds   1   

                     Match3                                        Match4  \
                       Home                 Away W/L                 Home   
1240_A       kashiwa-reysol             fc-tokyo   2  sanfrecce-hiroshima   
1240_B    kawasaki-frontale    consadole-sapporo   1          gamba-osaka   
1241_A   urawa-red-diamonds          vissel-kobe   1  yokohama-fa-marinos   
1242_A       vegalta-sendai         cerezo-osaka   0         oita-trinita   
1245_A  sanfrecce-hiroshima       kashiwa-reysol   1      kashima-antlers   
1246_A     tokushima-vortis  yokohama-fa-marinos   2      kashima-antlers   
1247_A       kashiwa-reysol  yokohama-fa-marinos   2  sanfrecce-hiroshima   
1248_A  sanfrecce-hiroshima          yokohama-fc   2     tokushima-vortis   

                                            Match5                            
                      Away W/L                Home                  Away W/L  
1240_A    tokushima-vortis   2        oita-trinita            sagan-tosu   0  
1240_B  urawa-red-diamonds   2         vissel-kobe          cerezo-osaka   0  
1241_A      kashiwa-reysol   0    tokushima-vortis  nagoya-grampus-eight   0  
1242_A      avispa-fukuoka   1         gamba-osaka           yokohama-fc   1  
1245_A      vegalta-sendai   0  urawa-red-diamonds       shonan-bellmare   2  
1246_A   consadole-sapporo   1            fc-tokyo          oita-trinita   1  
1247_A          sagan-tosu   0      vegalta-sendai    urawa-red-diamonds   0  
1248_A     shimizu-s-pulse   0     shonan-bellmare              fc-tokyo   2

In [141]:

# df=pd.DataFrame(columns=["No","Group","Date","Winning","Match1","Match2","Match3","Match4","Match5"])
# for n in range(1245,1250):
#     url =f'https://toto.rakuten.co.jp/toto/result/{n}/'
#     r = requests.get(url)
#     soup = BeautifulSoup(r.text, 'html.parser')
    
#     for group in ["A","B"]:
#         date = soup.select(f"#minitotoBlock{group} .tbl-basic-day td")[0].string
#         wining = soup.select(f"#minitotoBlock{group} .tbl-basic-money tbody td")[0].string
#         toto_data = [n, group, date,wining]

#         match = soup.select(f"#minitotoBlock{group} .tbl-result tbody tr")
#         for i in range(5):
#             match_datas =[match[i].select("td")[1].string, match[i].select("td")[3].string, match[i].select("td")[5].string]
#             toto_data.append(match_datas)

# #         print(toto_data)
#         df.loc[f"{n}_{group}"]= toto_data
    
    
#     time.sleep(1)


In [142]:
df["Match1"]

,Home,Away,W/L
1240_A,vegalta-sendai,avispa-fukuoka,2
1240_B,shimizu-s-pulse,nagoya-grampus-eight,2
1241_A,sagan-tosu,kashima-antlers,1
1242_A,yokohama-fa-marinos,shimizu-s-pulse,1
1245_A,avispa-fukuoka,vissel-kobe,2
1246_A,v-varen-nagasaki,jubilo-iwata,2
1247_A,matsumoto-yamaga,tokyo-verdy,1
1248_A,yokohama-fa-marinos,avispa-fukuoka,1


In [143]:
df

No Group           Date  Winning               Match1  \
                                                           Home   
1240_A  1240     A  2021年5月18日(火)  35,220円       vegalta-sendai   
1240_B  1240     B  2021年5月18日(火)   6,930円      shimizu-s-pulse   
1241_A  1241     A  2021年5月25日(火)  34,450円           sagan-tosu   
1242_A  1242     A   2021年6月1日(火)  11,700円  yokohama-fa-marinos   
1245_A  1245     A  2021年6月22日(火)  21,340円       avispa-fukuoka   
1246_A  1246     A  2021年6月29日(火)   1,970円     v-varen-nagasaki   
1247_A  1247     A   2021年7月6日(火)  13,800円     matsumoto-yamaga   
1248_A  1248     A  2021年7月13日(火)  34,790円  yokohama-fa-marinos   

                                           Match2                           \
                        Away W/L             Home                 Away W/L   
1240_A        avispa-fukuoka   2  kashima-antlers  yokohama-fa-marinos   1   
1240_B  nagoya-grampus-eight   2      yokohama-fc      shonan-bellmare   1   
1241_A       kashima-antlers   1   vegalta-sendai         oita-trinita   1   
1242_A       shimizu-s-pulse   1         fc-tokyo  sanfrecce-hiroshima   0   
1245_A           vissel-kobe   2      yokohama-fc             fc-tokyo   2   
1246_A          jubilo-iwata   2      yokohama-fc      shimizu-s-pulse   0   
1247_A           tokyo-verdy   1      vissel-kobe      shonan-bellmare   1   
1248_A        avispa-fukuoka   1     oita-trinita   urawa-red-diamonds   1   

                     Match3                                        Match4  \
                       Home                 Away W/L                 Home   
1240_A       kashiwa-reysol             fc-tokyo   2  sanfrecce-hiroshima   
1240_B    kawasaki-frontale    consadole-sapporo   1          gamba-osaka   
1241_A   urawa-red-diamonds          vissel-kobe   1  yokohama-fa-marinos   
1242_A       vegalta-sendai         cerezo-osaka   0         oita-trinita   
1245_A  sanfrecce-hiroshima       kashiwa-reysol   1      kashima-antlers   
1246_A     tokushima-vortis  yokohama-fa-marinos   2      kashima-antlers   
1247_A       kashiwa-reysol  yokohama-fa-marinos   2  sanfrecce-hiroshima   
1248_A  sanfrecce-hiroshima          yokohama-fc   2     tokushima-vortis   

                                            Match5                            
                      Away W/L                Home                  Away W/L  
1240_A    tokushima-vortis   2        oita-trinita            sagan-tosu   0  
1240_B  urawa-red-diamonds   2         vissel-kobe          cerezo-osaka   0  
1241_A      kashiwa-reysol   0    tokushima-vortis  nagoya-grampus-eight   0  
1242_A      avispa-fukuoka   1         gamba-osaka           yokohama-fc   1  
1245_A      vegalta-sendai   0  urawa-red-diamonds       shonan-bellmare   2  
1246_A   consadole-sapporo   1            fc-tokyo          oita-trinita   1  
1247_A          sagan-tosu   0      vegalta-sendai    urawa-red-diamonds   0  
1248_A     shimizu-s-pulse   0     shonan-bellmare              fc-tokyo   2

In [144]:
# df.set_index(["No","Date","Group"], drop=True)

In [145]:
df["Date"] = df["Date"].replace('\(.+\)','',regex=True)
df["Date"] = pd.to_datetime(df["Date"], format="%Y年%m月%d日")

In [146]:
df

No Group       Date  Winning               Match1  \
                                                       Home   
1240_A  1240     A 2021-05-18  35,220円       vegalta-sendai   
1240_B  1240     B 2021-05-18   6,930円      shimizu-s-pulse   
1241_A  1241     A 2021-05-25  34,450円           sagan-tosu   
1242_A  1242     A 2021-06-01  11,700円  yokohama-fa-marinos   
1245_A  1245     A 2021-06-22  21,340円       avispa-fukuoka   
1246_A  1246     A 2021-06-29   1,970円     v-varen-nagasaki   
1247_A  1247     A 2021-07-06  13,800円     matsumoto-yamaga   
1248_A  1248     A 2021-07-13  34,790円  yokohama-fa-marinos   

                                           Match2                           \
                        Away W/L             Home                 Away W/L   
1240_A        avispa-fukuoka   2  kashima-antlers  yokohama-fa-marinos   1   
1240_B  nagoya-grampus-eight   2      yokohama-fc      shonan-bellmare   1   
1241_A       kashima-antlers   1   vegalta-sendai         oita-trinita   1   
1242_A       shimizu-s-pulse   1         fc-tokyo  sanfrecce-hiroshima   0   
1245_A           vissel-kobe   2      yokohama-fc             fc-tokyo   2   
1246_A          jubilo-iwata   2      yokohama-fc      shimizu-s-pulse   0   
1247_A           tokyo-verdy   1      vissel-kobe      shonan-bellmare   1   
1248_A        avispa-fukuoka   1     oita-trinita   urawa-red-diamonds   1   

                     Match3                                        Match4  \
                       Home                 Away W/L                 Home   
1240_A       kashiwa-reysol             fc-tokyo   2  sanfrecce-hiroshima   
1240_B    kawasaki-frontale    consadole-sapporo   1          gamba-osaka   
1241_A   urawa-red-diamonds          vissel-kobe   1  yokohama-fa-marinos   
1242_A       vegalta-sendai         cerezo-osaka   0         oita-trinita   
1245_A  sanfrecce-hiroshima       kashiwa-reysol   1      kashima-antlers   
1246_A     tokushima-vortis  yokohama-fa-marinos   2      kashima-antlers   
1247_A       kashiwa-reysol  yokohama-fa-marinos   2  sanfrecce-hiroshima   
1248_A  sanfrecce-hiroshima          yokohama-fc   2     tokushima-vortis   

                                            Match5                            
                      Away W/L                Home                  Away W/L  
1240_A    tokushima-vortis   2        oita-trinita            sagan-tosu   0  
1240_B  urawa-red-diamonds   2         vissel-kobe          cerezo-osaka   0  
1241_A      kashiwa-reysol   0    tokushima-vortis  nagoya-grampus-eight   0  
1242_A      avispa-fukuoka   1         gamba-osaka           yokohama-fc   1  
1245_A      vegalta-sendai   0  urawa-red-diamonds       shonan-bellmare   2  
1246_A   consadole-sapporo   1            fc-tokyo          oita-trinita   1  
1247_A          sagan-tosu   0      vegalta-sendai    urawa-red-diamonds   0  
1248_A     shimizu-s-pulse   0     shonan-bellmare              fc-tokyo   2

In [147]:
df=df.set_index(["Date"], drop=True)

In [148]:
df

No Group  Winning               Match1                        \
                                                Home                  Away   
Date                                                                         
2021-05-18  1240     A  35,220円       vegalta-sendai        avispa-fukuoka   
2021-05-18  1240     B   6,930円      shimizu-s-pulse  nagoya-grampus-eight   
2021-05-25  1241     A  34,450円           sagan-tosu       kashima-antlers   
2021-06-01  1242     A  11,700円  yokohama-fa-marinos       shimizu-s-pulse   
2021-06-22  1245     A  21,340円       avispa-fukuoka           vissel-kobe   
2021-06-29  1246     A   1,970円     v-varen-nagasaki          jubilo-iwata   
2021-07-06  1247     A  13,800円     matsumoto-yamaga           tokyo-verdy   
2021-07-13  1248     A  34,790円  yokohama-fa-marinos        avispa-fukuoka   

                         Match2                                        Match3  \
           W/L             Home                 Away W/L                 Home   
Date                                                                            
2021-05-18   2  kashima-antlers  yokohama-fa-marinos   1       kashiwa-reysol   
2021-05-18   2      yokohama-fc      shonan-bellmare   1    kawasaki-frontale   
2021-05-25   1   vegalta-sendai         oita-trinita   1   urawa-red-diamonds   
2021-06-01   1         fc-tokyo  sanfrecce-hiroshima   0       vegalta-sendai   
2021-06-22   2      yokohama-fc             fc-tokyo   2  sanfrecce-hiroshima   
2021-06-29   2      yokohama-fc      shimizu-s-pulse   0     tokushima-vortis   
2021-07-06   1      vissel-kobe      shonan-bellmare   1       kashiwa-reysol   
2021-07-13   1     oita-trinita   urawa-red-diamonds   1  sanfrecce-hiroshima   

                                                  Match4                      \
                           Away W/L                 Home                Away   
Date                                                                           
2021-05-18             fc-tokyo   2  sanfrecce-hiroshima    tokushima-vortis   
2021-05-18    consadole-sapporo   1          gamba-osaka  urawa-red-diamonds   
2021-05-25          vissel-kobe   1  yokohama-fa-marinos      kashiwa-reysol   
2021-06-01         cerezo-osaka   0         oita-trinita      avispa-fukuoka   
2021-06-22       kashiwa-reysol   1      kashima-antlers      vegalta-sendai   
2021-06-29  yokohama-fa-marinos   2      kashima-antlers   consadole-sapporo   
2021-07-06  yokohama-fa-marinos   2  sanfrecce-hiroshima          sagan-tosu   
2021-07-13          yokohama-fc   2     tokushima-vortis     shimizu-s-pulse   

                            Match5                            
           W/L                Home                  Away W/L  
Date                                                          
2021-05-18   2        oita-trinita            sagan-tosu   0  
2021-05-18   2         vissel-kobe          cerezo-osaka   0  
2021-05-25   0    tokushima-vortis  nagoya-grampus-eight   0  
2021-06-01   1         gamba-osaka           yokohama-fc   1  
2021-06-22   0  urawa-red-diamonds       shonan-bellmare   2  
2021-06-29   1            fc-tokyo          oita-trinita   1  
2021-07-06   0      vegalta-sendai    urawa-red-diamonds   0  
2021-07-13   0     shonan-bellmare              fc-tokyo   2

In [149]:
df["2021-07-06":"2021-07-15"]

No Group  Winning               Match1                      \
                                                Home            Away W/L   
Date                                                                       
2021-07-06  1247     A  13,800円     matsumoto-yamaga     tokyo-verdy   1   
2021-07-13  1248     A  34,790円  yokohama-fa-marinos  avispa-fukuoka   1   

                  Match2                                       Match3  \
                    Home                Away W/L                 Home   
Date                                                                    
2021-07-06   vissel-kobe     shonan-bellmare   1       kashiwa-reysol   
2021-07-13  oita-trinita  urawa-red-diamonds   1  sanfrecce-hiroshima   

                                                  Match4                       \
                           Away W/L                 Home             Away W/L   
Date                                                                            
2021-07-06  yokohama-fa-marinos   2  sanfrecce-hiroshima       sagan-tosu   0   
2021-07-13          yokohama-fc   2     tokushima-vortis  shimizu-s-pulse   0   

                     Match5                          
                       Home                Away W/L  
Date                                                 
2021-07-06   vegalta-sendai  urawa-red-diamonds   0  
2021-07-13  shonan-bellmare            fc-tokyo   2

In [150]:
df["Winning"]=df["Winning"].str.replace(",","")
df["Winning"]=df["Winning"].str.replace("円","").astype(int)

In [151]:
df

No Group Winning               Match1                            \
                                               Home                  Away W/L   
Date                                                                            
2021-05-18  1240     A   35220       vegalta-sendai        avispa-fukuoka   2   
2021-05-18  1240     B    6930      shimizu-s-pulse  nagoya-grampus-eight   2   
2021-05-25  1241     A   34450           sagan-tosu       kashima-antlers   1   
2021-06-01  1242     A   11700  yokohama-fa-marinos       shimizu-s-pulse   1   
2021-06-22  1245     A   21340       avispa-fukuoka           vissel-kobe   2   
2021-06-29  1246     A    1970     v-varen-nagasaki          jubilo-iwata   2   
2021-07-06  1247     A   13800     matsumoto-yamaga           tokyo-verdy   1   
2021-07-13  1248     A   34790  yokohama-fa-marinos        avispa-fukuoka   1   

                     Match2                                        Match3  \
                       Home                 Away W/L                 Home   
Date                                                                        
2021-05-18  kashima-antlers  yokohama-fa-marinos   1       kashiwa-reysol   
2021-05-18      yokohama-fc      shonan-bellmare   1    kawasaki-frontale   
2021-05-25   vegalta-sendai         oita-trinita   1   urawa-red-diamonds   
2021-06-01         fc-tokyo  sanfrecce-hiroshima   0       vegalta-sendai   
2021-06-22      yokohama-fc             fc-tokyo   2  sanfrecce-hiroshima   
2021-06-29      yokohama-fc      shimizu-s-pulse   0     tokushima-vortis   
2021-07-06      vissel-kobe      shonan-bellmare   1       kashiwa-reysol   
2021-07-13     oita-trinita   urawa-red-diamonds   1  sanfrecce-hiroshima   

                                                  Match4                      \
                           Away W/L                 Home                Away   
Date                                                                           
2021-05-18             fc-tokyo   2  sanfrecce-hiroshima    tokushima-vortis   
2021-05-18    consadole-sapporo   1          gamba-osaka  urawa-red-diamonds   
2021-05-25          vissel-kobe   1  yokohama-fa-marinos      kashiwa-reysol   
2021-06-01         cerezo-osaka   0         oita-trinita      avispa-fukuoka   
2021-06-22       kashiwa-reysol   1      kashima-antlers      vegalta-sendai   
2021-06-29  yokohama-fa-marinos   2      kashima-antlers   consadole-sapporo   
2021-07-06  yokohama-fa-marinos   2  sanfrecce-hiroshima          sagan-tosu   
2021-07-13          yokohama-fc   2     tokushima-vortis     shimizu-s-pulse   

                            Match5                            
           W/L                Home                  Away W/L  
Date                                                          
2021-05-18   2        oita-trinita            sagan-tosu   0  
2021-05-18   2         vissel-kobe          cerezo-osaka   0  
2021-05-25   0    tokushima-vortis  nagoya-grampus-eight   0  
2021-06-01   1         gamba-osaka           yokohama-fc   1  
2021-06-22   0  urawa-red-diamonds       shonan-bellmare   2  
2021-06-29   1            fc-tokyo          oita-trinita   1  
2021-07-06   0      vegalta-sendai    urawa-red-diamonds   0  
2021-07-13   0     shonan-bellmare              fc-tokyo   2

In [152]:
df["No"]=df["No"].astype(int)

In [155]:
for i in range(1,6):
    try:
        df[f"Match{i}","W/L"]=df[f"Match{i}","W/L"].astype(int)
    except ValueError:
        pass

In [156]:
type(df.loc["2021-05-25"]["Match1"]["W/L"])

numpy.int64